In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import cv2
from os.path import join


def fetch(img_dir, name):
    img = cv2.imread(join(img_dir, name))
    if img.shape == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


def resize(img, size=(1024, 768)):
    assert len(size) == 2
    return cv2.resize(img, size, interpolation=cv2.INTER_CUBIC)


def pad(img, size=(1024, 768)):
    assert len(img.shape) == 3
    assert len(size) == 2
    h, w, _ = img.shape
    #assert w <= size[0] and h <= size[1]
    pad_vert = np.ceil((size[1]-h) / 2).astype(np.uint32)
    pad_hor = np.ceil((size[0]-w) / 2).astype(np.uint32)

    padded = np.full((size[1], size[0], 3), 255).astype(np.uint8)
    padded[pad_vert:pad_vert+h, pad_hor:pad_hor+w, :] = img.copy()
    return padded



## В GetItem есть место задания количества классов

In [2]:
import sys 
sys.path.insert(0, '../')
import numpy as np
import pandas as pd
from keras.utils import Sequence
from sklearn.utils import shuffle

from keras.utils import to_categorical

    
class WordsSequence(Sequence):
    def __init__(self, img_dir, input_shape, x_set, y_set=None, batch_size=1, classification=False):
        if classification:
            if y_set is not None:
                self.x, self.y = x_set, y_set
                self.dataset = pd.DataFrame(data={'x': self.x, 'y': self.y, 'used': np.zeros_like(self.y)})
            else:
                self.x, self.y = x_set, None
        else:
            if y_set is not None:
                self.x, self.y = x_set, y_set
                self.x, self.y = shuffle(self.x, self.y)
            else:
                self.x, self.y = x_set, None

        self.img_dir = img_dir
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.classification = classification


    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        if self.classification:
            if self.y is None:
                batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
                return np.array([self.preprocess(fetch(self.img_dir, name)) for name in batch_x])

            unused = self.dataset.loc[self.dataset['used'] == 0]
            if len(unused) >= self.batch_size:
                batch_indices = unused.sample(n=self.batch_size).index
            else:
                batch_indices = unused.sample(n=self.batch_size, replace=True).index

            self.dataset.loc[batch_indices, 'used'] = 1
            batch_x = self.dataset.iloc[batch_indices]['x'].values
            batch_y = self.dataset.iloc[batch_indices]['y'].values 
            ##########################
            # Здесь вместо 108 - количество классов
            return np.array([self.preprocess(fetch(self.img_dir, name)) for name in batch_x]), to_categorical(batch_y, 108)

        if self.y is None:
            x = self.x[idx]
            return np.expand_dims(self.preprocess(fetch(self.img_dir, x)), axis=0)


        curr_x = self.x[idx]
        curr_y = self.y[idx]

        x_1_images = self.preprocess(fetch(self.img_dir, curr_x[0]))
        x_2_images = self.preprocess(fetch(self.img_dir, curr_x[1]))
        return [np.expand_dims(x_1_images, axis=0), np.expand_dims(x_2_images, axis=0)], np.array([curr_y])


       

    def preprocess(self, img):
        assert len(img.shape) == 3

        h, w, _ = img.shape
        if h / w <= self.input_shape[0] / self.input_shape[1]:
            img = resize(img, (self.input_shape[1], int(self.input_shape[1] * h / w)))
        else:
            img = resize(img, (int(self.input_shape[0] * w / h), self.input_shape[0]))

        img = pad(img, (self.input_shape[1], self.input_shape[0]))
        return img / 255. 

    def on_epoch_end(self):
        if (not self.classification) and (self.y is not None):
            self.x, self.y = shuffle(self.x, self.y)
        
        if self.classification and self.y is not None:
            self.dataset = pd.DataFrame(data={'x': self.x, 'y': self.y, 'used': np.zeros_like(self.y)})
            self.dataset = self.dataset.sample(n=len(self.dataset))


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import os
import numpy as np
import pandas as pd
from collections import Counter



from keras import backend as K
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.applications.mobilenet import MobileNet
from keras.layers import Input, Dense, Lambda, Dropout, Activation
from keras.optimizers import rmsprop, Adam, SGD, Adagrad


from keras.utils import to_categorical
from keras.models import Model


def get_str2numb_numb2dict(vect):
    str_to_ind_dict = {}
    count = 0
    for v in vect:
        if v not in str_to_ind_dict.keys():
            str_to_ind_dict[v] = count
            count += 1
    reverse_dict = {v:k for k, v in str_to_ind_dict.items()}
    return str_to_ind_dict, reverse_dict

    
def apply_dict(dict_keys, X):
    res = []
    for x in X:
        if x in dict_keys.keys():
            res.append(dict_keys[x])
        else:
            res.append(-1)
    return res
    
    
class Classification_model:
    def __init__(self, alpha, input_shape, num_classes, cache_dir, train_head, batch_size=32):
        self.alpha = alpha
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.cache_dir = cache_dir
        self.batch_size = batch_size
        if not os.path.isdir(self.cache_dir):
            os.makedirs(self.cache_dir)
        self.build_model(train_head)
    
    def build_model(self, train_head):        
        # If imagenet weights are being loaded, alpha can be one of`0.25`, `0.50`, `0.75` or `1.0` only.
        base_model = MobileNet(input_shape=self.input_shape, alpha=self.alpha, weights='imagenet', 
                               include_top=False, pooling='avg') 
        
        # Use to pretrain head
        # if train_head:
            # for layer in base_model.layers[:-4]:
                # layer.trainable = False
            
        # base_model.summary()
        op = Dense(128, activation='relu')(base_model.output)
        op = Dropout(0.00001)(op)
        output_tensor = Dense(self.num_classes, activation='softmax')(op)
       
        self.model = Model(inputs=base_model.input, outputs=output_tensor)
        self.model.summary()
        
    def train(self, train_dir, train_csv, epochs, learning_rate=0.00001):
        train = pd.read_csv(train_csv)
        train_x, train_y = train['file_name'].as_matrix(), train['label'].as_matrix()
        
        self.str2ind_dict, self.ind2str_dict = get_str2numb_numb2dict(train_y)
        train_y = np.array(apply_dict(self.str2ind_dict, train_y))

        train_generator = WordsSequence(img_dir=train_dir,
                                        input_shape = self.input_shape,
                                        x_set=train_x,
                                        y_set=train_y,
                                        batch_size=self.batch_size,
                                        classification=True)
                                        
        optimize = rmsprop(lr=learning_rate, decay=1e-6)
        # optimize = Adam(lr=0.00000001) 
        # optimize = SGD()
        # optimize = Adagrad(lr=0.0001)
        
        self.model.compile(loss='categorical_crossentropy', optimizer=optimize, metrics=['categorical_accuracy'])
        
        self.model.fit_generator(train_generator,
                                 steps_per_epoch=len(train_x)//self.batch_size,
                                 shuffle=True,
                                 epochs=epochs,
                                 verbose=1, 
                                 callbacks=[ModelCheckpoint(filepath=file_path_to_checkpoint, save_weights_only=True)])
        path_to_save_model = self.cache_dir + '/' + 'final_model.h5'
        path_to_save_weights = self.cache_dir + '/' + 'final_weights.h5'
        
    def save_weights(self, filename):
        self.model.save_weights(filename)
        
    def load_weights(self, filename):
        self.model.load_weights(filename, by_name=True, skip_mismatch=True)
        
    def predict(self, test_dir, test_csv): 

        test = pd.read_csv(test_csv)
        test_x, test_y = test['file_name'].as_matrix(), test['label'].as_matrix()
        self.str2ind_dict, self.ind2str_dict = get_str2numb_numb2dict(test_y)
        test_generator = WordsSequence(img_dir=test_dir,
                                        input_shape = self.input_shape,
                                        x_set=test_x,
                                        batch_size=self.batch_size,
                                        classification=True)
                                        
        pred = np.argmax(self.model.predict_generator(test_generator, verbose=1), axis=1)  
        res = np.array(apply_dict(self.ind2str_dict, pred))
        
        count = 0
        for i,j in zip(res, test_y):
            if i == j:
                count += 1
        print('word accuracy: ', count / len(test_y))
        
        count = 0
        autors = np.unique(test_y)
        autor_ind = [np.argwhere(test_y == a) for a in autors]
        for i,inds in enumerate(autor_ind):
            p = Counter(np.ravel(res[inds])).most_common(1)[0][0]
            if p == autors[i]:
                count += 1

        print('top-1 autor accuracy: ', count / len(autors))
        
        сount = 0
        for i,inds in enumerate(autor_ind):
            p = [pair[0] for pair in Counter(np.ravel(res[inds])).most_common(5)]
            if autors[i] in p:
                сount += 1

        print('top-5 autor accuracy: ', сount / len(autors))


In [4]:
cache_dir = 'Literature data/classification_cache'
train_dir = 'Literature data/train_set'
test_dir = 'Literature data/test_set'

# Train
model = Classification_model(alpha=1, input_shape=(160,160,3), num_classes=108, cache_dir=cache_dir, train_head=False)




Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 160, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 161, 161, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 80, 80, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 80, 80, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 80, 80, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 80, 80, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 80, 80, 32)        128 

In [5]:
train_csv = 'Literature data/train.csv'
train = pd.read_csv(train_csv)
train_x, train_y = train['file_name'].as_matrix(), train['label'].as_matrix()

model.str2ind_dict, model.ind2str_dict = get_str2numb_numb2dict(train_y)


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
self = model
train_y = np.array(apply_dict(self.str2ind_dict, train_y))

In [7]:
train_y

array([  0,   0,   0, ..., 107, 107, 107])

In [8]:
train_generator = WordsSequence(img_dir=train_dir,
                                input_shape = self.input_shape,
                                x_set=train_x,
                                y_set=train_y,
                                batch_size=self.batch_size,
                                classification=True)
train_generator

In [9]:
learning_rate=0.00001
optimize = rmsprop(lr=learning_rate, decay=1e-6)
# optimize = Adam(lr=0.00000001) 
# optimize = SGD()
# optimize = Adagrad(lr=0.0001)
optimize

In [10]:

self.model.compile(loss='categorical_crossentropy', optimizer=optimize, metrics=['categorical_accuracy'])


In [11]:
epochs = 100
file_path_to_checkpoint=self.cache_dir + '/checkpoint-{epoch:02d}.h5'

In [12]:
 self.model.fit_generator(train_generator,
                         steps_per_epoch=len(train_x)//self.batch_size,
                          shuffle=True,
                         epochs=epochs,
                         verbose=1)

Epoch 1/100
115/115 [==============================] - 865s 8s/step - loss: 4.7992 - categorical_accuracy: 0.0196
Epoch 2/100
115/115 [==============================] - 838s 7s/step - loss: 4.5306 - categorical_accuracy: 0.0495
Epoch 3/100
115/115 [==============================] - 810s 7s/step - loss: 4.3348 - categorical_accuracy: 0.0720
Epoch 4/100
115/115 [==============================] - 828s 7s/step - loss: 4.1549 - categorical_accuracy: 0.1106
Epoch 5/100
115/115 [==============================] - 842s 7s/step - loss: 4.0097 - categorical_accuracy: 0.1345
Epoch 6/100
115/115 [==============================] - 806s 7s/step - loss: 3.8433 - categorical_accuracy: 0.1704
Epoch 7/100
115/115 [==============================] - 840s 7s/step - loss: 3.6963 - categorical_accuracy: 0.2046
Epoch 8/100
115/115 [==============================] - 850s 7s/step - loss: 3.5416 - categorical_accuracy: 0.2424
Epoch 9/100
115/115 [==============================] - 838s 7s/step - loss: 3.3925 - cat

In [14]:
path_model = 'Literature Data/classification_cache/my_model.h5'
self.model.save(path_model)

In [16]:
path_to_save_weights = self.cache_dir + '/' + 'final_weights.h5'
self.save_weights(path_to_save_weights)

In [17]:
test_dir = 'Literature data/test_set'
test_csv = 'Literature data/test.csv'
self.predict(test_dir, test_csv)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:109: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


86/86 [==============================] - 236s 3s/step
word accuracy:  0.16072727272727272
top-1 autor accuracy:  0.35185185185185186
top-5 autor accuracy:  0.4722222222222222


In [18]:
test = pd.read_csv(test_csv)
test_x, test_y = test['file_name'].as_matrix(), test['label'].as_matrix()
self.str2ind_dict, self.ind2str_dict = get_str2numb_numb2dict(test_y)
test_generator = WordsSequence(img_dir=test_dir,
                                input_shape = self.input_shape,
                                x_set=test_x,
                                batch_size=self.batch_size,
                                classification=True)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [19]:
test_y

array(['1_0', '1_0', '1_0', ..., '7_9', '7_9', '7_9'], dtype=object)

In [20]:
pred = np.argmax(self.model.predict_generator(test_generator, verbose=1), axis=1)  
res = np.array(apply_dict(self.ind2str_dict, pred))


86/86 [==============================] - 227s 3s/step


In [21]:
count = 0
autors = np.unique(test_y)
autors

array(['1_0', '1_1', '1_10', '1_12', '1_13', '1_14', '1_15', '1_16',
       '1_17', '1_18', '1_19', '1_2', '1_20', '1_21', '1_22', '1_23',
       '1_24', '1_3', '1_4', '1_5', '1_6', '1_8', '1_9', '2_0', '2_1',
       '2_10', '2_11', '2_12', '2_13', '2_14', '2_15', '2_2', '2_3',
       '2_4', '2_5', '2_6', '2_8', '2_9', '3_0', '3_1', '3_11', '3_12',
       '3_13', '3_14', '3_15', '3_16', '3_17', '3_18', '3_19', '3_2',
       '3_3', '3_4', '3_5', '3_6', '3_8', '3_9', '4_1', '4_11', '4_15',
       '4_17', '4_18', '4_19', '4_2', '4_20', '4_21', '4_22', '4_3',
       '4_4', '4_5', '4_6', '4_7', '4_8', '4_9', '5_0', '5_1', '5_10',
       '5_11', '5_12', '5_13', '5_14', '5_15', '5_16', '5_17', '5_18',
       '5_2', '5_9', '6_0', '6_10', '6_11', '6_12', '6_6', '6_7', '6_8',
       '6_9', '7_0', '7_1', '7_10', '7_11', '7_12', '7_13', '7_2', '7_3',
       '7_4', '7_5', '7_6', '7_7', '7_8', '7_9'], dtype=object)

In [22]:
autor_ind = [np.argwhere(test_y == a) for a in autors]
autor_ind

[array([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15],
        [16],
        [17]], dtype=int64), array([[165],
        [166],
        [167],
        [168],
        [169],
        [170],
        [171],
        [172],
        [173],
        [174],
        [175],
        [176],
        [177],
        [178],
        [179]], dtype=int64), array([[18],
        [19],
        [20],
        [21],
        [22],
        [23],
        [24],
        [25],
        [26],
        [27],
        [28],
        [29]], dtype=int64), array([[30],
        [31],
        [32],
        [33],
        [34],
        [35],
        [36],
        [37],
        [38],
        [39],
        [40]], dtype=int64), array([[41],
        [42],
        [43],
        [44],
        [45],
        [46],
        [47],
        [48],
        [49],
        [50],
     

In [28]:
count = 0
for i,inds in enumerate(autor_ind):
    p = Counter(np.ravel(res[inds])).most_common(1)[0][0]
    if autors[i] in p :
        print('совпал автор №  ' + str(autors[i]))
        count += 1
    else:
        print('не совпал автор №   ' + str(autors[i]) + ' не совпало с ' + str(p))

print('top-1 autor accuracy: ', count / len(autors))


совпал автор №  1_0
совпал автор №  1_1
не совпал автор №   1_10 не совпало с 7_9
не совпал автор №   1_12 не совпало с 6_10
совпал автор №  1_13
совпал автор №  1_14
совпал автор №  1_15
не совпал автор №   1_16 не совпало с 4_17
совпал автор №  1_17
не совпал автор №   1_18 не совпало с 7_0
совпал автор №  1_19
не совпал автор №   1_2 не совпало с 3_15
совпал автор №  1_20
совпал автор №  1_21
совпал автор №  1_22
совпал автор №  1_23
совпал автор №  1_24
не совпал автор №   1_3 не совпало с 4_6
совпал автор №  1_4
не совпал автор №   1_5 не совпало с 3_18
не совпал автор №   1_6 не совпало с 3_18
совпал автор №  1_8
совпал автор №  1_9
не совпал автор №   2_0 не совпало с 6_9
совпал автор №  2_1
совпал автор №  2_10
совпал автор №  2_11
не совпал автор №   2_12 не совпало с 6_12
совпал автор №  2_13
совпал автор №  2_14
не совпал автор №   2_15 не совпало с 4_5
совпал автор №  2_2
совпал автор №  2_3
совпал автор №  2_4
не совпал автор №   2_5 не совпало с 3_5
совпал автор №  2_6
не

In [29]:
count = 0
for i,inds in enumerate(autor_ind):
    p = [pair[0] for pair in Counter(np.ravel(res[inds])).most_common(5)]
    if autors[i] in p:
        print('совпал автор №  ' + str(autors[i]))
        count += 1
    else:
        print('не совпал автор №   ' + str(autors[i]) + ' не совпало с ' + str(p))

print('top-5 autor accuracy: ', count / len(autors))

совпал автор №  1_0
совпал автор №  1_1
не совпал автор №   1_10 не совпало с ['7_9', '7_11', '2_8', '4_22', '7_7']
не совпал автор №   1_12 не совпало с ['6_10', '4_18', '3_13', '6_9', '3_5']
совпал автор №  1_13
совпал автор №  1_14
совпал автор №  1_15
не совпал автор №   1_16 не совпало с ['4_17', '3_18', '5_18', '4_2', '2_1']
совпал автор №  1_17
совпал автор №  1_18
совпал автор №  1_19
совпал автор №  1_2
совпал автор №  1_20
совпал автор №  1_21
совпал автор №  1_22
совпал автор №  1_23
совпал автор №  1_24
не совпал автор №   1_3 не совпало с ['4_6', '5_12', '5_17', '6_11', '5_0']
совпал автор №  1_4
не совпал автор №   1_5 не совпало с ['3_18', '4_6', '7_2', '6_8', '4_9']
совпал автор №  1_6
совпал автор №  1_8
совпал автор №  1_9
совпал автор №  2_0
совпал автор №  2_1
совпал автор №  2_10
совпал автор №  2_11
совпал автор №  2_12
совпал автор №  2_13
совпал автор №  2_14
совпал автор №  2_15
совпал автор №  2_2
совпал автор №  2_3
совпал автор №  2_4
совпал автор №  2_5
сов

In [30]:
count = 0
for i,j in zip(res, test_y):
    if i == j:
        count += 1
        print(i)
print('word accuracy: ', count / len(test_y))

1_0
1_0
1_0
1_0
1_0
1_0
1_0
1_0
1_0
1_0
1_0
1_13
1_13
1_13
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_14
1_15
1_15
1_15
1_15
1_15
1_15
1_15
1_15
1_16
1_17
1_17
1_17
1_17
1_17
1_17
1_17
1_17
1_18
1_18
1_19
1_19
1_19
1_19
1_19
1_1
1_1
1_1
1_1
1_1
1_1
1_1
1_20
1_20
1_20
1_20
1_20
1_20
1_20
1_20
1_20
1_20
1_21
1_21
1_21
1_22
1_22
1_22
1_22
1_23
1_23
1_23
1_23
1_23
1_24
1_24
1_2
1_2
1_3
1_4
1_4
1_4
1_4
1_4
1_4
1_5
1_6
1_6
1_6
1_6
1_8
1_8
1_8
1_8
1_8
1_9
1_9
1_9
1_9
2_0
2_0
2_0
2_0
2_10
2_10
2_10
2_11
2_11
2_11
2_11
2_11
2_11
2_12
2_12
2_12
2_12
2_12
2_13
2_13
2_13
2_13
2_14
2_14
2_14
2_14
2_14
2_14
2_14
2_14
2_14
2_15
2_15
2_15
2_15
2_15
2_15
2_15
2_15
2_15
2_15
2_1
2_1
2_1
2_1
2_1
2_1
2_1
2_1
2_1
2_1
2_1
2_1
2_2
2_2
2_2
2_2
2_2
2_2
2_2
2_2
2_2
2_3
2_3
2_3
2_3
2_3
2_3
2_3
2_3
2_3
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_4
2_5
2_5
2_5
2_5
2_5
2_6
2_6
2_6
2_6
2_8


### Называется - сверяйся с данными не хочу, а я и не хочу. Проверять итерацию в test.csv и фотку в test_set

In [33]:
for i , word in enumerate(res):
    print('iteration ' + str(i) + ' - ' + str(word))

iteration 0 - 1_0
iteration 1 - 1_20
iteration 2 - 1_0
iteration 3 - 1_0
iteration 4 - 3_15
iteration 5 - 1_20
iteration 6 - 1_0
iteration 7 - 1_0
iteration 8 - 3_15
iteration 9 - 1_0
iteration 10 - 1_0
iteration 11 - 1_0
iteration 12 - 1_0
iteration 13 - 1_0
iteration 14 - 1_0
iteration 15 - 6_12
iteration 16 - 6_12
iteration 17 - 1_17
iteration 18 - 2_8
iteration 19 - 7_11
iteration 20 - 4_22
iteration 21 - 7_9
iteration 22 - 7_11
iteration 23 - 7_9
iteration 24 - 7_7
iteration 25 - 7_9
iteration 26 - 3_1
iteration 27 - 5_13
iteration 28 - 3_3
iteration 29 - 5_0
iteration 30 - 3_13
iteration 31 - 6_9
iteration 32 - 3_5
iteration 33 - 6_10
iteration 34 - 3_0
iteration 35 - 1_18
iteration 36 - 5_11
iteration 37 - 6_10
iteration 38 - 4_18
iteration 39 - 1_14
iteration 40 - 4_18
iteration 41 - 1_13
iteration 42 - 1_23
iteration 43 - 3_18
iteration 44 - 6_12
iteration 45 - 4_18
iteration 46 - 6_12
iteration 47 - 3_14
iteration 48 - 6_12
iteration 49 - 1_13
iteration 50 - 3_17
iteration 51

iteration 1540 - 4_6
iteration 1541 - 4_20
iteration 1542 - 3_4
iteration 1543 - 3_4
iteration 1544 - 4_21
iteration 1545 - 4_21
iteration 1546 - 7_10
iteration 1547 - 1_3
iteration 1548 - 3_14
iteration 1549 - 3_14
iteration 1550 - 2_1
iteration 1551 - 4_18
iteration 1552 - 1_18
iteration 1553 - 4_6
iteration 1554 - 1_5
iteration 1555 - 4_19
iteration 1556 - 5_13
iteration 1557 - 4_21
iteration 1558 - 1_5
iteration 1559 - 1_8
iteration 1560 - 4_21
iteration 1561 - 1_20
iteration 1562 - 7_10
iteration 1563 - 4_21
iteration 1564 - 4_21
iteration 1565 - 4_21
iteration 1566 - 1_4
iteration 1567 - 1_1
iteration 1568 - 4_21
iteration 1569 - 1_5
iteration 1570 - 1_24
iteration 1571 - 5_18
iteration 1572 - 3_4
iteration 1573 - 2_3
iteration 1574 - 2_3
iteration 1575 - 3_12
iteration 1576 - 2_13
iteration 1577 - 7_10
iteration 1578 - 3_12
iteration 1579 - 2_4
iteration 1580 - 3_12
iteration 1581 - 4_22
iteration 1582 - 6_11
iteration 1583 - 3_15
iteration 1584 - 3_0
iteration 1585 - 3_3
iterat

In [13]:
import matplotlib.pylab as plt
def show(img):
    """
    show rgb image
    """
    ax = plt.axes([0,0,4,4], frameon=False)
    ax.set_axis_off()
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [12]:
size=(1024, 768)
count = 0
test_dir = 'Literature data/test_set'
imgs = os.listdir(test_dir)
for j, img_p in enumerate(imgs):
    path = test_dir + '/' + img_p
    img = cv2.imread(path)
    assert len(img.shape) == 3
    assert len(size) == 2
    h, w, _ = img.shape
    if w > size[0] or h > size[1]:
        #os.remove(path)
        count += 1
print(count)


0


In [ ]:
import cv2
import numpy as np
#import image
image = cv2.imread('input.jpg')
#cv2.imshow('orig',image)
#cv2.waitKey(0)

#grayscale
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow('gray',gray)
cv2.waitKey(0)

#binary
ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
cv2.imshow('second',thresh)
cv2.waitKey(0)

#dilation
kernel = np.ones((5,5), np.uint8)
img_dilation = cv2.dilate(thresh, kernel, iterations=1)
cv2.imshow('dilated',img_dilation)
cv2.waitKey(0)

#find contours
im2,ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#sort contours
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

for i, ctr in enumerate(sorted_ctrs):
    # Get bounding box
    x, y, w, h = cv2.boundingRect(ctr)

    # Getting ROI
    roi = image[y:y+h, x:x+w]

    # show ROI
    cv2.imshow('segment no:'+str(i),roi)
    cv2.rectangle(image,(x,y),( x + w, y + h ),(90,0,255),2)
    cv2.waitKey(0)

cv2.imshow('marked areas',image)
cv2.waitKey(0)

In [122]:
def pad(img, size=(1024, 768)):
    assert len(img.shape) == 3
    assert len(size) == 2
    h, w, _ = img.shape
    #assert w <= size[0] and h <= size[1]
    if w <= size[0] and h <= size[1]:
        pad_vert = np.ceil((size[1]-h) / 2).astype(np.uint32)
        pad_hor = np.ceil((size[0]-w) / 2).astype(np.uint32)
        padded = np.full((size[1], size[0], 3), 255).astype(np.uint8)
        padded[pad_vert:pad_vert+h, pad_hor:pad_hor+w, :] = img.copy()
    elif  w <= size[0] and h > size[1]:
        pad_vert = np.ceil(math.fabs((size[1]-h)) / 2).astype(np.uint32)
        pad_hor = np.ceil((size[0]-w) / 2).astype(np.uint32)
        padded = np.full((size[1], size[0], 3), 255).astype(np.uint8)
        padded[ pad_vert:h-pad_vert, pad_hor:pad_hor+w, :] = img.copy()
    elif w > size[0] and h <= size[1]:
        pad_vert = np.ceil((size[1]-h) / 2).astype(np.uint32)
        pad_hor = np.ceil(math.fabs((size[0]-w)/2) ).astype(np.uint32)
        print(pad_vert)
        print(pad_hor)
        padded = np.full((size[1], size[0], 3), 255).astype(np.uint8)
        padded[pad_vert:pad_vert+h, 0 :size[0] , :] = img.copy()
    else:
        pad_vert = np.ceil(math.fabs((size[1]-h)) / 2).astype(np.uint32)
        pad_hor = np.ceil(math.fabs((size[0]-w)) / 2).astype(np.uint32)
        padded = np.full((size[1], size[0], 3), 255).astype(np.uint8)
        padded[pad_vert:h-pad_vert, pad_hor :size[0] + pad_hor, :] = img.copy() 
    print(pad_hor)
    print(pad_vert)
    return padded